#1

**Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;**

In [94]:
#Importacoes necessarias
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

#Leitura do Arquivo CSV
results = pd.read_csv('https://raw.githubusercontent.com/BielMaracaja/RI/master/ri_lab_06/results.csv')
#results = results.astype(str)
documents = results['text']

tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
index = {}
n = 0

for text in documents:
  palavras = [palavra for palavra in tokenizador.tokenize(text.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  
  for j in palavras:
    if j not in index.keys():
      index[j] = []
    index[j].append(n)
    
for element in index.items():
  d = dict(collections.Counter(element[1]))
  index[element[0]] = list(d.items())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


#2
**Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário;**

In [95]:
T = results.text.count()

for palavra in index:
  k = len(index[palavra])
  idf = round(np.log((T+1)/k), 2)
  index[palavra].append(idf)

print (index)

 

{'juíza': [(1, 2), (2, 1), 4.83], 'federal': [(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, 1), (24, 1), (34, 1), (37, 2), (42, 1), (50, 1), (113, 1), (115, 2), (126, 1), (134, 1), (138, 1), (139, 2), (144, 1), (151, 5), (162, 2), (165, 1), (166, 1), (168, 2), (171, 2), (173, 4), (178, 1), (202, 1), (203, 2), (204, 1), (205, 1), (206, 2), (207, 2), (208, 2), (210, 1), (213, 2), (214, 3), (217, 2), (219, 1), (220, 3), (223, 1), (224, 1), (225, 1), (226, 1), (228, 2), (229, 3), (233, 1), (236, 1), (239, 2), (248, 3), 1.63], 'ivani': [(1, 1), (2, 1), 4.83], 'silva': [(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73, 1), (76, 1), (115, 1), (183, 1), (226, 1), (234, 1), (236, 1), 3.04], 'luz': [(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78, 1), (79, 1), (81, 1), (103, 1), (119, 1), (125, 1), (130, 1), (147, 1), (154, 1), (170, 1), (177, 1), (179, 1), (204, 1), (224, 1), (249, 1), 2.53], 'brasília': [(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (48, 1), (50, 1), (53, 1), (62, 1), (72, 1), (80, 1),

#3
**Implementar as seguintes versões do modelo vetorial:**



**.1 Representação binária;**

In [0]:
def vsm_binario(query, document):
  pontuacao = 0
  
  query_tokens = query.split()
  
  tokens_doc = document.split()
  
  for token in query_tokens:
    pontuacao += (token in tokens_doc)
  
  return pontuacao

**.2 TF (lembre-se que esse modelo já está implementado);**

In [0]:
def vsm_tf(query, document):
  pontuacao = 0
  
  tokens_doc = document.split()
  
  query_tokens = query.split()
  
  for palavra in query_tokens:
    pontuacao += tokens_doc.count(palavra)
  
  return pontuacao

**.3 TF-IDF;**

In [0]:
def vsm_tdif(query, document):
  pontuacao = 0
  
  tokens_doc  = document.split()
  
  query_tokens = query.split()
  
  for palavra in query_tokens:
    cwd = tokens_doc.count(palavra)
    pontuacao+= cwd * index[palavra][-1]
  
  return round(pontuacao,2)

**.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho).**

In [0]:
def vsm_bm25(query, document, k):
  pontuacao = 0
  
  tokens_doc = document.split()
  
  query_tokens = query.split()
  
  palavras = [palavra for palavra in query_tokens if palavra in tokens_doc]
    
  for palavra in palavras:
    cwd = tokens_doc.count(palavra)
    dfw = len(index[palavra][:-1])
    pontuacao += (((k+1) * cwd) / (cwd + k)) * np.log10(((T+1) / dfw))
  return round(pontuacao,2)

#4
**Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;**

In [0]:
listens = ['educação', 'reforma', 'bolsonaro']

In [0]:
def top_5_create_models(listen):
  count = 0
  
  db = []
  
  dtf = []
  
  dtfidf = []
  
  dbm25 = []
  
  for doc in results.text:
    doc = doc.lower()
    count += 1
  
    bisect.insort(db, (vsm_binario(listen, doc), count))
    bisect.insort(dtf, (vsm_tf(listen,doc), count))
    bisect.insort(dtfidf, (vsm_tdif(listen,doc), count))
    bisect.insort(dbm25, (vsm_bm25(listen,doc,10), count))
  
  db.reverse()
  dtf.reverse()
  
  dtfidf.reverse()
  dbm25.reverse()
  
  return db[:5], dtf[:5], dtfidf[:5], dbm25[:5]

In [0]:
binario_top_5 = ['','','']

tf_top_5 = ['','','']

tf_top_5idf = ['','','']

top5_bm25 = ['','','']

binario_top_5[0], tf_top_5[0], tf_top_5idf[0], top5_bm25[0] = top_5_create_models(listens[0])

binario_top_5[1], tf_top_5[1], tf_top_5idf[1], top5_bm25[1] = top_5_create_models(listens[1])

binario_top_5[2], tf_top_5[2], tf_top_5idf[2], top5_bm25[2] = top_5_create_models(listens[2])

In [103]:
listen_df = pd.DataFrame()
listen_df['Listens'] = listens

listen_df['Binary'] = binario_top_5
listen_df['TF'] = tf_top_5

listen_df['TF-IDF'] = tf_top_5idf
listen_df['BM25'] = top5_bm25

listen_df.index+=1
listen_df

,Listens,Binary,TF,TF-IDF,BM25
1,educação,"[(1, 248), (1, 240), (1, 239), (1, 233), (1, 2...","[(15, 221), (8, 222), (6, 239), (6, 130), (5, ...","[(32.85, 221), (17.52, 222), (13.14, 239), (13...","[(6.28, 221), (4.65, 222), (3.92, 239), (3.92,..."
2,reforma,"[(1, 248), (1, 240), (1, 236), (1, 235), (1, 2...","[(8, 138), (6, 205), (6, 37), (5, 166), (4, 248)]","[(16.96, 138), (12.72, 205), (12.72, 37), (10....","[(4.5, 138), (3.8, 205), (3.8, 37), (3.38, 166..."
3,bolsonaro,"[(1, 248), (1, 240), (1, 238), (1, 237), (1, 2...","[(32, 151), (30, 207), (30, 166), (19, 19), (1...","[(42.24, 151), (39.6, 207), (39.6, 166), (25.0...","[(4.79, 151), (4.72, 207), (4.72, 166), (4.12,..."


#5
**Compare os resultados encontrados e responda.**

**.1 Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.**

In [0]:
def top1_get_score(top5):
  return [top[0][0] for top in top5]
def top1_get_doc(top5):
  return [top[0][1] for top in top5]
def doc_get_title(docs):
  return [results.title[doc] for doc in docs]

binario_top5_score = top1_get_score(binario_top_5)

tf_top5_score = top1_get_score(tf_top_5)

tfidf_top5_score = top1_get_score(tf_top_5idf)

bm25_top5_score = top1_get_score(top5_bm25)

binario_top5_doc = top1_get_doc(binario_top_5)

tf_top5_doc = top1_get_doc(tf_top_5)

tfidf_top5_doc = top1_get_doc(tf_top_5idf)

bm25_top5_doc = top1_get_doc(top5_bm25)

titulo_binario = doc_get_title(binario_top5_doc)

titulo_tf = doc_get_title(tf_top5_doc)

titulo_tfidf = doc_get_title(tfidf_top5_doc)

titulo_bm25 = doc_get_title(bm25_top5_doc)



In [105]:
#Conceito Binario

binary_res = pd.DataFrame()

binary_res['Procurar'] = listens
binary_res['Numero Documento'] = binario_top5_doc
binary_res['Titulo Documento'] = titulo_binario
binary_res['Pontuacao Binaria'] = binario_top5_score
binary_res.index += 1

binary_res

,Procurar,Numero Documento,Titulo Documento,Pontuacao Binaria
1,educação,248,As três espanholas do Estado Islâmico: “Só que...,1
2,reforma,248,As três espanholas do Estado Islâmico: “Só que...,1
3,bolsonaro,248,As três espanholas do Estado Islâmico: “Só que...,1


In [106]:
#Conceito TF

tf_res = pd.DataFrame()

tf_res['Procurar'] = listens
tf_res['Numero Documento'] = tf_top5_doc
tf_res['Titulo Documento'] = titulo_tf
tf_res['Pontuacao TF'] = tf_top5_score
tf_res.index += 1

tf_res

,Procurar,Numero Documento,Titulo Documento,Pontuacao TF
1,educação,221,A jovem deputada que jogou contra as cordas o ...,15
2,reforma,138,Estatal espanhola fica com aeroportos do Norde...,8
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,32


In [107]:
#Conceito TF-IDF

tfidf_res = pd.DataFrame()

tfidf_res['Procurar'] = listens
tfidf_res['Numero Documento'] = tfidf_top5_doc
tfidf_res['Titulo Documento'] = titulo_tfidf
tfidf_res['Pontuacao TFIDF'] = tfidf_top5_score
tfidf_res.index += 1

tfidf_res

,Procurar,Numero Documento,Titulo Documento,Pontuacao TFIDF
1,educação,221,A jovem deputada que jogou contra as cordas o ...,32.85
2,reforma,138,Estatal espanhola fica com aeroportos do Norde...,16.96
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,42.24


In [108]:
#Conceito BM-25

bm25_res = pd.DataFrame()

bm25_res['Procurar'] = listens
bm25_res['Numero Documento'] = bm25_top5_doc
bm25_res['Titulo Documento'] = titulo_bm25
bm25_res['Pontuacao BM-25'] = bm25_top5_score
bm25_res.index += 1

bm25_res

,Procurar,Numero Documento,Titulo Documento,Pontuacao BM-25
1,educação,221,A jovem deputada que jogou contra as cordas o ...,6.28
2,reforma,138,Estatal espanhola fica com aeroportos do Norde...,4.50
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,4.79


De acordo com os resultados mostrados logo acima podemos ver que o TF-IDF e o BM-25 exibem resultados próximos e também nos apresentam a nós os melhores resultados.

Na busca "educação", temos a saida:

    "A jovem deputada que jogou contra as cordas o ..."

Na busca "reforma", temos a saida:

    "Estatal espanhola fica com aeroportos do Norde..."

Na busca "bolsonaro", temos a saida:

    "Socialismo ‘millennial’ nos EUA"
    
Ao final podemos notar que, de acordo com a complexidade do algoritmo TF-IDF, mesmo sendo simples, dá a nós um resultado descente.

**.2 Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)**

In [109]:
def inter_size(a,b):
  return len([elem for elem in a if elem in b])

def jaccard_index(a,b):
  n_a = len(a)
  n_b = len(b)
  n_ab = inter_size(a,b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

all_docs = []
all_docs.append(binario_top5_doc)
all_docs.append(tf_top5_doc)
all_docs.append(tfidf_top5_doc)
all_docs.append(bm25_top5_doc)

matrix = [['Medida', 'Binario', 'TF', 'TF-IDF', 'BM25']]
titles = matrix
for i in range(len(all_docs)):
  line = []
  for j in range(len(all_docs)):
    jaccard = jaccard_index(all_docs[i], all_docs[j])
    line.append(round(jaccard,2))
  matrix.append(line)
for i in range(1, len(matrix)):
  matrix[i].insert(0, matrix[0][i])
  
pd.DataFrame(matrix)

,0,1,2,3,4
0,Medida,Binario,TF,TF-IDF,BM25
1,Binario,1,0,0,0
2,TF,0,1,1,1
3,TF-IDF,0,1,1,1
4,BM25,0,1,1,1


De acordo com o índice de Jaccard a diferênça do modelo binário para os demais é bem aparente, porque o mesmo não tem similaridade com nenhum outro. Os modelos TF-Idf e BM-25, podemos notar o maior valor do índice de Jaccard, o qual mostra igualdade entre eles, de maneira que acaba nos mostrando porque o TF-IDF é o melhor modelo. Voltando a questão anterior, notamos que o modelo TF mostra resultados não satisfatórios para determinada consulta. 